In [1]:
using MyBioTools.FluTools
using Dates
using Plots, StatsPlots
using StatsBase, Statistics
using TreeTools
using Profile, ProfileView
using BenchmarkTools
using BioTools, BioSequences
using Random

In [5]:
sp = FluTools.StrainPop("../data/aligned_h3n2_ha_aa.fasta");
FluTools.remove_gapped_strains!(sp);
FluTools.bin_by_date!(sp, start = Date(2002,12,1), binwidth=Day(61), binspacing = Day(122));
datebins = sort(collect(keys(sp.datebin)));

Removing strains...


In [17]:
# Only rerun for A NEW SELECTION OF STRAINS
NEW_STRAIN_SELECTION = false
if NEW_STRAIN_SELECTION
    strains_to_write = Array{BioTools.Strain}(undef, 0)
    for db in datebins[1:end]
        pop = sp.datebin[db]
        strains = shuffle(pop)[1:min(100,length(pop))]
        cons = FluTools.consensus(strains)

        for s in strains
            dat = Dict()
            dat["strain"] = s.strain
            dat["virus"] = "flu"
            dat["date"] = s.date
    #         dat["lbi"] = round(s.fitness, digits=5) # Will be added when the tree is built
            dat["dist_to_consensus"] = FluTools.hamming(s.aa_seq, cons.aa_seq) #/ dist_to_cons_scale, digits=5)
            push!(strains_to_write, BioTools.Strain(s.aa_seq, dat, :aa))
        end
    end
end

In [41]:
# This was using AA strains. To construct the tree, we need dna sequences
nt_strains = BioTools.readfastastrains("../data/aligned_h3n2_ha.fasta", :dna, BioTools.augur_all_header_fields, strainfilters = [BioTools.gapfilter]);
BioTools.remove!(nt_strains, FluTools.outliers["h3n2"], verbose=true);

Reading ../data/aligned_h3n2_ha.fasta...
Read 46527 strains out of 61848. Filtered 15318. Could not read 3
Filtered 330 strains


In [18]:
# Getting the nt sequence of all strains to write
labels = map(x->x.data["strain"], strains_to_write)
nt_strains_to_write = Dict{String, Strain{BioSequences.DNAAlphabet{4}}}()
for st in nt_strains
    tmp = findfirst(==(st.data["strain"]), labels)
    if !isnothing(tmp) && st.data["country"]!="?"
        nt_strains_to_write[st.data["strain"]] = Strain(st.seq, merge(strains_to_write[tmp].data, st.data))
    end
end

In [19]:
# Writing them to fasta
fields = [:strain]#, :date, :virus, :region, :country, :authors, :dist_to_consensus]
BioTools.writefasta("../data/newtrees/100_per_4month/secondary_files/aligned_ha_nolbi.fasta", collect(values(nt_strains_to_write)), fields)

# Once the raw tree is built
We will now compute LBI values, distance to consensus, and add this to metadata.

In [21]:
t = read_tree("../data/newtrees/100_per_4month/results/tree_raw.nwk", NodeDataType=LBIData);
FluTools.get_lbi!(sp, t);
for (d,v) in sp.lbi_datebin
    for s in values(sp.datebin[d])
        sp.straindict[s.strain].fitness = v[s.strain]
    end
end

  0.789511 seconds (1.89 M allocations: 170.036 MiB, 9.29% gc time)


In [26]:
# At this stage, `strains_to_write` may have strains with unknown country which 
# we filtered in `nt_strains_to_write`
# For this reasonm `strains_to_write` is updated here with a new variable
aa_strains_to_write = Dict{String, Strain{BioSequences.AminoAcidAlphabet}}()
for (n,s) in nt_strains_to_write
    s.data["lbi"] = sp.straindict[s.data["strain"]].fitness
    aa_strains_to_write[s.data["strain"]] = Strain(LongAminoAcidSeq(sp.straindict[s.data["strain"]].aa_seq), s.data)
end

In [27]:
fields = [:strain, :date, :virus, :region, :country, :authors, :dist_to_consensus, :lbi]
BioTools.writefasta("../data/newtrees/100_per_4month/aligned_ha_nt.fasta", collect(values(nt_strains_to_write)), fields)


In [28]:
?writefasta

search: writefasta write_fasta



```
writefasta([io::IO = stdout], data)
```

This version of the function writes to an already opened `IO` stream, defaulting to `stdout`.

---

```
writefasta(filename::String, data, [mode::String = "w"])
```

This function dumps data to a FASTA file, auto-formatting it so to follow the specifications detailed in the section titled [The FASTA format](@ref). The `data` can be anything which is iterable and which produces `(description, sequence)` tuples upon iteration, where the `description` must be convertible to a `String` and the `sequence` can be any iterable object which yields elements convertible to ASCII characters (e.g. a `String`, a `Vector{UInt8}` etc.).

Examples:

```julia
writefasta("somefile.fasta", [("GENE1", "GCATT"), ("GENE2", "ATTAGC")])
writefasta("somefile.fasta", ["GENE1" => "GCATT", "GENE2" => "ATTAGC"])
```

If the `filename` ends with `.gz`, the result will be a gzip-compressed file.

The `mode` flag determines how the `filename` is open; use `"a"` to append the data to an existing file.

---

```
writefasta(s::AbstractStrain, fields; fillvals = false)
writefasta(f::IO, s::AbstractStrain, fields; fillvals=false)
writefasta(S::Array{<:AbstractStrain}, fields; fillvals = false)
writefasta(f::String, S::Array{<:AbstractStrain}, fields; fillvals=false, mode="w")
```

Write strain `s` to a fasta format. Header is built using `fields` from `s.data`. 


In [32]:
# For compat with FluTools, I put specific fields for the AA sequences
aa_fields = ["strain", "virus", "accession", "date", "region", "country", "division", "location", "passage_category", "submitting_lab", "age", "gender"]
BioTools.writefasta("../data/newtrees/100_per_4month/aligned_ha_aa.fasta", collect(values(aa_strains_to_write)), aa_fields, fillvals = true)